# LightGBM Method

__Our third method will be a Light Gradient Boosting Machine__

This model was originaly created by Microsoft. It basically is a bunch of many small decision trees, build one after another, where each tree tries to correct the mistakes of the preious ones. 

We are going to trin a LightGBM model (Poisson) on the dataset,, we will evaluate with Poisson deviance, RMSE/MAE, and show calliration.

__First the libraries__

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

__Then we load and clean the data__

In [ ]:
train = pd.read_csv("../data/claims_train.csv")
test = pd.read_csv("../data/claims_test.csv")

print("Train shape:", train.shape)
train.head()

for col in train.columns:
    if train[col].dtype == "object":
        train[col] = train[col].fillna("Missing")#important gang, this is for categorical
    else:
        train[col] = train[col].fillna(train[col].median())#same stuff but for numerical twin


for col in test.columns:#same shit as before but with the test dataset
    if test[col].dtype == "object":
        test[col] = test[col].fillna("Missing")
    else:
        test[col] = test[col].fillna(test[col].median())



target = "ClaimNb"
exposure = "Exposure"

features = [c for c in train.columns if c not in [target, exposure]]

print("Features used:", features)

X = train[features]
y = train[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Train shape: (542410, 12)
Features used: ['IDpol', 'Area', 'VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region']


__Now we code the poisson deviance and evaluation metrics__

In [13]:
def poisson_deviance(y_true, y_pred, eps=1e-9):
    """
    Poisson devince for count data.
    y_true: observed counts
    y_pred: predicted (what we expect) counts (more or equal to 0)
    """
    y_pred = np.maximum(y_pred, eps)
    y_true = np.asarray(y_true)
    term = y_true * np.log((y_true + eps) / y_pred) - (y_true - y_pred)

    return 2.0 * np.mean(term)

def evaluate_counts(y_true, y_pred, exposure = None):
    """
    Evaluate predicted counts vs true counts.
    IF exposure is provided, predictons/true sould be counts.
    Return dict whit RMSE, MAE, Poisson deviance, total observed vs predicted.
    
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    pdev = poisson_deviance(y_true, y_pred)
    total_true = np.sum(y_true) if exposure is None else np.sum(y_true)
    total_pred = np.sum(y_pred) if exposure is None else np.sum(y_pred)
    return {"rmse": rmse, "mae": mae, "poisson_deviance": pdev,
            "total_true": total_true, "total_pred": total_pred}


In [15]:
train = pd.read_csv("../data/claims_train.csv")
train_path = '../data/claims_train.csv'
test = pd.read_csv("../data/claims_test.csv")

assert os.path.exists(train_path), f"Train file not found at {train_path}"
train_df = pd.read_csv(train)
print("Train shape:", train_df.shape)
train_df.head()
print(train_df.info())
print(train_df.describe(include='all').T[['count','unique','top','freq']].head(10))

print("ClaimNb distribution:")
print(train_df['ClaimNb'].value_counts().sort_index().head(10))
print("Exposure summary:")
print(train_df['Exposure'].describe())

TypeError: argument of type 'method' is not iterable